# End-to-end run no GPU

### Imports

In [5]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [6]:
# -- dev imports
# TODO prediction string stasrt from 0 or 1?
%load_ext autoreload
%autoreload 2

from argminer.data import ArgumentMiningDataset
from argminer.evaluation import inference, get_word_labels

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
path_label_map = '../../data/UCL/dataset2/df_label_map_bio.json'
path_dataset = '../../data/UCL/dataset2/df_labels_general.json'

df_label_map = pd.read_json(path_label_map)
df_text = pd.read_json(path_dataset)
df_text.predictionString = df_text.predictionString.apply(lambda x: ' '.join([str(item) for item in x]))
df_text.label_bio = df_text.label_bio.apply(lambda x: ' '.join([str(item) for item in x]))
df_text.head()

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]


In [8]:
df_input = df_text.groupby('doc_id').agg({
    'text':lambda x: ' '.join(x),
    'predictionString': lambda x: ' '.join(x),
    'label_bio': lambda x: ' '.join(x)
})
df_input.label_bio = df_input.label_bio.str.split()
df_input

,text,predictionString,label_bio
doc_id,,,
essay001,Should students be taught to compete or to coo...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay002,More people are migrating to other countries t...,388 389 390 391 392 393 394 395 396 397 398 39...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay003,International tourism is now more common than ...,734 735 736 737 738 739 740 741 742 743 744 74...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay004,International tourism is now more common than ...,1004 1005 1006 1007 1008 1009 1010 1011 1012 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay005,Living and studying overseas\n\nIt is every st...,1303 1304 1305 1306 1307 1308 1309 1310 1311 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
essay398,We can not forcedly put the same numbers of ma...,135846 135847 135848 135849 135850 135851 1358...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay399,"Drugs, alcohol and messy sex lives\n\nCelebrit...",136187 136188 136189 136190 136191 136192 1361...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay400,A greater proportion of the budget should be a...,136459 136460 136461 136462 136463 136464 1364...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [9]:
df_input.label_bio.values[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'I-MajorClaim',
 'O',
 'O',
 'O',
 'O',
 'B-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'I-Claim',
 'O',
 'B-Premise',
 'I-Premise',
 'I-Premise',
 'I-Premise',
 'I-Premise',
 'I-Premise',
 'I-Prem

In [4]:
df_input

NameError: name 'df_input' is not defined

In [28]:
model_name = 'google/bigbird-roberta-base'
num_labels = df_label_map.shape[0]

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_name)
optimizer = torch.optim.Adam(params=model.parameters())


Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [88]:
limit_size = 2

dataset = ArgumentMiningDataset(
    df_label_map,
    df_input[['text', 'label_bio']].rename(columns={'label_bio':'labels'}),
    tokenizer,
    max_length=256,
    strategy='standard_bio'
)

trainloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [102]:
epochs = 2
for epoch in range(epochs):
    training_loss = 0
    for i, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()

        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
            
        training_loss += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break
    print(training_loss/(i+1))

1.6381123065948486
1.5479151010513306


In [130]:
val_dataset = ArgumentMiningDataset(
    df_label_map,
    df_input[['text', 'label_bio']].rename(columns={'label_bio':'labels'}),
    tokenizer,
    max_length=512,
    strategy='standard_bio',
    is_train = False
)
valloader = DataLoader(val_dataset, batch_size=2, shuffle=False)
print(df_input)

                                                       text  \
doc_id                                                        
essay001  Should students be taught to compete or to coo...   
essay002  More people are migrating to other countries t...   
essay003  International tourism is now more common than ...   
essay004  International tourism is now more common than ...   
essay005  Living and studying overseas\n\nIt is every st...   
...                                                     ...   
essay398  We can not forcedly put the same numbers of ma...   
essay399  Drugs, alcohol and messy sex lives\n\nCelebrit...   
essay400  A greater proportion of the budget should be a...   
essay401  Fatherhood should be as present as motherhood ...   
essay402  Children should studying hard or playing sport...   

                                           predictionString  \
doc_id                                                        
essay001  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18

In [213]:
def evaluate(df_outputs, df_targets):
    # TODO problem of duplicates? Think about this with test cases
    """
    Calculates the F-Score given prediction strings
    """
    
    # -- Constants
    gt, pred = '_gt', '_pred'
    
    #    Merge constants
    merge_on = ['id', 'class']
    suffixes = (gt, pred)

    #    Columns labels
    gt_id = f'index{gt}'
    pred_id = f'index{pred}'
    
    overlap_pred_gt = f'overlap{pred}{gt}'  # how much of ground truth in prediction
    overlap_gt_pred = f'overlap{gt}{pred}'  # how much of prediction in ground truth
    pred_col_name = 'predictionString'
    gt = f'{pred_col_name}{gt}'
    pred = f'{pred_col_name}{pred}'


    
    # TODO is this even correct?? Would you not be repeating items??
    # find all combinations
    # REally really not sure of this assumption..
    
    df_targets = df_targets.reset_index()
    df_outputs = df_outputs.reset_index()
    
    df_targets = df_targets.merge(
        df_outputs, on=merge_on, how='outer', suffixes=suffixes
    )
    
    df_targets[[gt, pred]] = df_targets[[gt, pred]].where(~df_targets[[gt, pred]].isnull(), {-1})

    # find intersection and normalise against each item
    df_targets[overlap_pred_gt] = df_targets[[gt, pred]].apply(
        lambda x: len(x[gt].intersection(x[pred])) / len(x[pred]), axis=1
    )
    
    df_targets[overlap_gt_pred] = df_targets[[pred, gt]].apply(
        lambda x: len(x[gt].intersection(x[pred])) / len(x[gt]), axis=1
    )
    
    
    df_targets['tp'] = df_targets[[overlap_pred_gt, overlap_gt_pred]].apply(
        lambda x: int(x.overlap_pred_gt >= 0.5 and x.overlap_gt_pred >= 0.5), axis=1
    )
    df_targets['overlap_max'] = df_targets[[overlap_pred_gt, overlap_gt_pred]].max(axis=1)

    df_targets[pred] = df_targets[pred].apply(lambda x: tuple(x))
    df_targets[gt] = df_targets[gt].apply(lambda x: tuple(x))
    
    # only keep the ones with the highest overall score
    
    
    # I'm not convinced by this tbh? What about cases of perfect match on one but not on the other?
    # e.g. case when you have 0.5 and 1 and then 0.8 and 1
    df_targets_match = df_targets[df_targets['tp'] == 1].sort_values('overlap_max', ascending=False).groupby(
        ['id', 'class', gt] # group by pred or gt?
    ).first()

    TP = df_targets_match.groupby('class')[pred_id].nunique().to_frame('tp').reset_index()
    
    df_false_positive = df_targets[
        # they didn't do this filter?
        #(df_targets['tp'] == 0) & 
        (~df_targets.set_index(pred_id).index.isin(df_targets_match[pred_id]))
        
    ]
    
    FP = df_false_positive.groupby('class')[pred_id].nunique().to_frame('fp').reset_index()


    matched_gt_id = df_targets[df_targets['tp'] == 1][gt_id].unique()
    
    print(df_targets[df_targets[gt_id].isin(matched_gt_id)].sort_values('class'))

    df_false_negative = df_targets[
        #(df_targets['tp'] == 0) & 
        (~df_targets.set_index(gt_id).index.isin(matched_gt_id))
    ]
    
    FN = df_false_negative.groupby('class')[gt_id].nunique().to_frame('fn').reset_index()
    
    #TP = df_targets_match.reset_index().groupby(['id', 'class']).size().to_frame('tp').reset_index()
    #FN = df_false_negatives.groupby(['id', 'class']).size().to_frame('fn').reset_index()
    #FP = df_false_positives.groupby(['id', 'class']).size().to_frame('fp').reset_index()
    
    scores = TP.merge(
        FN.merge(FP, how='outer'), how='outer'
    ).fillna(0)
    scores = scores.assign(f1=lambda x: x['tp'] / (x['tp'] + 1/2 * (x['fp'] + x['fn'])))
    print(scores)


    return scores['f1'].mean()


In [214]:
df_outputs = pd.DataFrame(data={
    'id':[1,1,2,1, 1, 3, 3, 4],
    'class':[1,1,1, 2, 1, 3, 3, 4],
    'predictionString':[{1,2}, {6,7}, {1,2,3}, {1,2}, {4,5,6,7}, {16,17}, {18,19}, {1}] # TODO WHAT HAPPENS WITH THIS?
})
df_targets = pd.DataFrame(data={
    'id':[1,1,1,2, 1,3],
    'class':[1,1,1,1, 2,3],
    'predictionString':[{1,2,3, 4, 5}, {6,7,8}, {21,22,23,24,25}, {1,2,3}, {1,2}, {16,17,18,19}]
})

print(evaluate(df_outputs, df_targets))

    index_gt  id  class predictionString_gt  index_pred predictionString_pred  \
3        1.0   1      1           (8, 6, 7)           0                (1, 2)   
4        1.0   1      1           (8, 6, 7)           1                (6, 7)   
5        1.0   1      1           (8, 6, 7)           4          (4, 5, 6, 7)   
9        3.0   2      1           (1, 2, 3)           2             (1, 2, 3)   
10       4.0   1      2              (1, 2)           3                (1, 2)   
11       5.0   3      3    (16, 17, 18, 19)           5              (16, 17)   
12       5.0   3      3    (16, 17, 18, 19)           6              (18, 19)   

    overlap_pred_gt  overlap_gt_pred  tp  overlap_max  
3               0.0         0.000000   0     0.000000  
4               1.0         0.666667   1     1.000000  
5               0.5         0.666667   1     0.666667  
9               1.0         1.000000   1     1.000000  
10              1.0         1.000000   1     1.000000  
11             

In [221]:
def calc_overlap(row):
    """
    计算 prediction and ground truth的重叠部分
    """
    set_pred = set(row.new_predictionstring_pred.split(' '))
    set_gt = set(row.new_predictionstring_gt.split(' '))
    # length of each end intersection
    len_pred = len(set_pred)
    len_gt = len(set_gt)
    intersection = len(set_gt.intersection(set_pred))  # 两者交集
    overlap_1 = intersection / len_gt # 交集占ground truth比例
    overlap_2 = intersection / len_pred # 交集占prediction比例
    return [overlap_1, overlap_2]

def score_feedback_comp(pred_df, gt_df):
    
    """
    A function that scores for the kaggle
        Student Writing Competition
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    关于评价指标的介绍，详见讲义的第一周。
    """
    pred_df.new_predictionstring = pred_df.new_predictionstring.apply(
        lambda x: ' '.join([str(item) for item in x])
    )
    gt_df.new_predictionstring = gt_df.new_predictionstring.apply(
        lambda x: ' '.join([str(item) for item in x])
    )
    
    gt_df = gt_df[['id', 'discourse_type', 'new_predictionstring']].reset_index(drop = True).copy() # ground truth
    pred_df = pred_df[['id', 'class', 'new_predictionstring']].reset_index(drop = True).copy() # prediction
    gt_df['gt_id'] = gt_df.index
    pred_df['pred_id'] = pred_df.index
    joined = pred_df.merge( 
        gt_df, 
        left_on = ['id', 'class'],
        right_on = ['id', 'discourse_type'], 
        how = 'outer', # outer join
        suffixes = ['_pred', '_gt']  # 重复列的后缀
    )
    joined['new_predictionstring_gt'] =  joined['new_predictionstring_gt'].fillna(' ') # 填充空格
    joined['new_predictionstring_pred'] =  joined['new_predictionstring_pred'].fillna(' ') # 填充空格
    joined['overlaps'] = joined.apply(calc_overlap, axis = 1) # 
    # overlap over 0.5: true positive
    # If nultiple overlaps exists, the higher is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])
 
    # 两种overlap都大于0.5，则为TP
    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1', 'overlap2']].max(axis = 1) # 取大的值

    # query: 只取'potential_TP'==True的样本
    # sort_values: 按max_overlap降序
    # groupby: 按['id', 'new_predictionstring_gt'] 分组获取index
    tp_pred_ids = joined.query('potential_TP').sort_values('max_overlap', ascending = False)\
                  .groupby(['id', 'new_predictionstring_gt']).first()['pred_id'].values
    
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids] # FP数据
    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    print(joined[joined.gt_id.isin(matched_gt_ids)])
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids] # FN数据

    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    print(TP, FN, FP)
    # macro_f1_score 公式
    macro_f1_score = TP / (TP + 1/2 * (FP + FN))
    return macro_f1_score

import numpy as np
def oof_score(df_val, oof):
    '''
    OOF分数计算
    '''
    f1score = []
    classes = [1,2,3,4]
    # 计算每个class的分数
    for c in classes: 
        pred_df = oof.loc[oof['class'] == c].copy()
        gt_df = df_val.loc[df_val['discourse_type'] == c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(f'{c:<10}: {f1:4f}')
        f1score.append(f1)
    f1avg = np.mean(f1score)
    return f1avg

oof_score(
    df_targets.rename(columns={
        'class':'discourse_type',
        'predictionString': 'new_predictionstring'
    }),
    df_outputs.rename(columns={
        'predictionString': 'new_predictionstring'
    })
)

   id  class new_predictionstring_pred  pred_id  discourse_type  \
1   1      1                       1 2        0               1   
4   1      1                       6 7        1               1   
7   1      1                   4 5 6 7        3               1   
9   2      1                     1 2 3        2               1   

  new_predictionstring_gt  gt_id                   overlaps  overlap1  \
1                   8 6 7      1                 [0.0, 0.0]  0.000000   
4                   8 6 7      1  [0.6666666666666666, 1.0]  0.666667   
7                   8 6 7      1  [0.6666666666666666, 0.5]  0.666667   
9                   1 2 3      3                 [1.0, 1.0]  1.000000   

   overlap2  potential_TP  max_overlap  
1       0.0         False     0.000000  
4       1.0          True     1.000000  
7       0.5          True     0.666667  
9       1.0          True     1.000000  
2 2 2
1         : 0.500000
   id  class new_predictionstring_pred  pred_id  discourse_type  \

0.5416666666666666

In [196]:
with torch.no_grad():
    
    for i, (inputs, targets) in enumerate(valloader):
        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
        
        # validation should have this, all can still be on the GPU
        word_ids = inputs['word_ids']
        doc_ids = inputs['index']
        
        word_label_probas = get_word_labels(word_ids, outputs, agg_strategy='first', has_x=False)
        # can still be GPU
        word_labels = [tensor.argmax(dim=1) for tensor in word_label_probas]
        # still GPU
        
        target_labels = get_word_labels(word_ids, targets, agg_strategy='first', has_x=False)
        
        
        # todo need to have a label to map groups of items back to each other,
        # so first convert back to strings, then relabel based on strings, e.g. remove the 'B-I' stuff!
        df_targets_predString = get_predictionString(target_labels, doc_ids)
        df_outputs_predString = get_predictionString(word_labels, doc_ids)
        
        evaluate(df_outputs_predString, df_targets_predString)
        
        
        # also needs the doc ID for the mapping, so doc_id is missing!!!!
        break

NameError: name 'valloader' is not defined

In [140]:
pd.DataFrame({'tp':[True,True,False, False], 'fp':[1,2,3,4]}).query('tp')

,tp,fp
0,True,1
1,True,2


In [144]:
df_text[df_text.doc_id == 'essay001']

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]
5,T4,Premise,What we acquired from team work is not only ho...,716,851,essay001,116 117 118 119 120 121 122 123 124 125 126 12...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
6,O,Other,.,851,853,essay001,142,O,[O]
7,T5,Premise,"During the process of cooperation, children ca...",853,1086,essay001,143 144 145 146 147 148 149 150 151 152 153 15...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
8,O,Other,.,1086,1088,essay001,180,O,[O]
9,T6,Premise,All of these skills help them to get on well w...,1088,1191,essay001,181 182 183 184 185 186 187 188 189 190 191 19...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."


In [157]:
def get_predictionString(labels, doc_ids):
    """ 
    """
    # TODO need to ensure that we are mapping to the correct class names and doc IDs!
    ids = []
    classes = []
    prediction_strings = []
    for doc_id, label in zip(doc_ids, labels):
        unique_classes, unique_class_counts = torch.unique_consecutive(label, return_counts=True)
        start_id = 1
        for unique_class, unique_class_count in zip(unique_classes, unique_class_counts):
            ids.append(doc_id.item())
            classes.append(unique_class.item())
            end_id = start_id + unique_class_count + 1 
            prediction_strings.append(set(range(start_id, end_id)))
            start_id = end_id
    return pd.DataFrame(data={'id': ids, 'class': classes, 'predictionString': prediction_strings})

In [127]:
df_text[df_text.doc_id == 'essay001']

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]
5,T4,Premise,What we acquired from team work is not only ho...,716,851,essay001,116 117 118 119 120 121 122 123 124 125 126 12...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
6,O,Other,.,851,853,essay001,142,O,[O]
7,T5,Premise,"During the process of cooperation, children ca...",853,1086,essay001,143 144 145 146 147 148 149 150 151 152 153 15...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
8,O,Other,.,1086,1088,essay001,180,O,[O]
9,T6,Premise,All of these skills help them to get on well w...,1088,1191,essay001,181 182 183 184 185 186 187 188 189 190 191 19...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."


In [ ]:
def get_predictionString(labels, doc_ids):
    for label in labels:
        unique_classes, unique_class_counts = torch.unique_consecutive(return_counts=True)
        

In [99]:
df_input.iloc[[0,1]].label_bio.apply(lambda x: len(x))

doc_id
essay001    387
essay002    346
Name: label_bio, dtype: int64

,0,1
0,1,2
1,3,4
